In [17]:
%pip install --quiet -U azureml-fsspec dotenv
%cd Users/djenk.ivanov/


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[Errno 2] No such file or directory: 'Users/djenk.ivanov/'
/mnt/batch/tasks/shared/LS_root/mounts/clusters/movielens-compute/code/Users/djenk.ivanov


In [18]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from dotenv import load_dotenv
import os

load_dotenv()

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_ML_WORKSPACE"),
)

In [38]:
import pandas as pd

data_asset = ml_client.data.get(name="movielens32m", version="1")
print(f'Data asset path: {data_asset.path}')

ratings = pd.read_csv(data_asset.path + "/ratings.csv")
movies = pd.read_csv(data_asset.path + "/movies.csv")

Data asset path: azureml://subscriptions/ba117730-305b-41b8-8447-bf6834f0a56f/resourcegroups/djenk-rg1/workspaces/djenk-ml/datastores/movielens32m_ds/paths/ml-32m/


In [20]:
ratings.head() 

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [21]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [49]:
hot_encode_genres = pd.Series(movies['genres']).str.get_dummies(sep='|')
encoded_movies = movies.merge(hot_encode_genres, left_index=True, right_index=True)
encoded_movies.drop(columns=['genres'], inplace=True)
encoded_movies.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
ratings = ratings.drop(columns=['timestamp'])
merged = ratings.merge(encoded_movies, on='movieId', how='left')

In [51]:
print(ratings.shape, movies.shape, merged.shape)
merged.head()

(32000204, 3) (87585, 3) (32000204, 24)


,userId,movieId,rating,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,17,4.0,Sense and Sensibility (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,25,1.0,Leaving Las Vegas (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,29,2.0,"City of Lost Children, The (Cité des enfants p...",0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,1,30,5.0,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,32,5.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0


In [94]:
import numpy as np

df = merged.copy()

df['liked'] = (df['rating'] >= 3).astype('int8')

row_hash = pd.util.hash_pandas_object(df[['userId', 'movieId', 'liked']], index=False).astype('uint64')
rand = (row_hash % (2**32)) / (2**32)

df['split'] = np.where(rand < 0.5, 'train', 'test')

In [ ]:
train = df[df['split'] == 'train']
test = df[df['split'] == 'test']

In [97]:
train.head()

,userId,movieId,rating,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,liked,split
2,1,29,2.0,"City of Lost Children, The (Cité des enfants p...",0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,train
5,1,34,2.0,Babe (1995),0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,train
6,1,36,1.0,Dead Man Walking (1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
10,1,161,1.0,Crimson Tide (1995),0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,train
11,1,166,5.0,"Doom Generation, The (1995)",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,train
